<h1>Growth Regularisation</h1>
This notebook demonstrates the implementation of this paper <a href=https://arxiv.org/abs/2012.09243> Neural Pruning Via Growth Regularisation</a>
<h4>Steps to train a baseline model and then compress it given a channel budget are as follows:</h4>
<ul>
    <li>Load the YAML file. </li>
    <li>Load dataset and create dataloaders. </li>
    <li>Create <b>Growth Regularisation</b> object and pass the parameters in the form of a dictionary. </li>
    <li>Pass the dataloaders into the <b>compress_model</b> method to obtain the compressed model. </li>
</ul>
Since this is a demo notebook the number of epochs have been set to 2.

In [1]:
import sys
import os
sys.path.append("../../")
sys.path.append("../../../")
os.environ['CUDA_VISIBLE_DEVICES']='2'

import torch
from torchvision import transforms

import yaml

from trailmet.datasets.classification import DatasetFactory
from trailmet.algorithms.prune.growth_regularisation import Growth_Regularisation

In [2]:
!pwd
root = "/workspace/code/Nahush26/trail/trailmet/experiments/pruning/growth_regularisation"

/workspace/code/Nahush26/trail/trailmet/experiments/pruning/growth_regularisation


<h3> Loading the YAML file. </h3>

In [3]:
with open(os.path.join(root, "resnet50_cifar100.yaml"), 'r') as stream:
    data_loaded = yaml.safe_load(stream)
data_loaded

{'num_classes': 100,
 'weight_decay': 0.0005,
 'arch': 'resnet50',
 'dataset': 'cifar100',
 'epochs': 1,
 'learning_rate': 0.002,
 'prune_ratio': 0.5,
 'reg_upper_limit': 0.0004,
 'reg_granularity_prune': 0.0002,
 'method': 'GReg-1'}

<h3>Loading CIFAR100Dataset</h3>

In [4]:
transform_train = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Pad(4, padding_mode='reflect'),
                    transforms.RandomHorizontalFlip(p=0.5),
                    transforms.RandomCrop(32),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
                ]
            )

transform_test = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
                ]
            )

transforms1 = {
    'train': transform_train, 
    'val': transform_test, 
    'test': transform_test}

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}

cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR100', 
                                        root = "./data",
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.1,
                                        transform = transforms1,
                                        target_transform = target_transforms,
                                        random_seed=42
                                        )

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


<h5>Creating the dataloaders</h5>

In [5]:
dataloaders = { 'train' : torch.utils.data.DataLoader(
        cifar_dataset['train'], batch_size=64, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=0
    ),
               'val':  torch.utils.data.DataLoader(
        cifar_dataset['val'], batch_size=64, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=0
    ),  
               'test':  torch.utils.data.DataLoader(
        cifar_dataset['test'], batch_size=64, 
        sampler=cifar_dataset['test_sampler'],
        num_workers=0
    )}

<h3> Creating the method's object proceed with compression. </h3>

In [6]:
slim = Growth_Regularisation()

cd /workspace/code/Nahush26/trail/trailmet/experiments/pruning/growth_regularisation
CUDA_VISIBLE_DEVICES=2 python /opt/conda/lib/python3.8/site-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-22910f2f-f08c-477b-8851-4948dcc52014.json

ExpNote [-utils-194418]: " " -- TrAIL


In [ ]:
slim.compress_model(dataloaders)

EPOCH NUMBER  0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:04<00:00, 16.39it/s, acc=0.0242, loss=8.88]


**Saving model**
Register layer index and kernel shape:
[ 0]                 conv1 -- kernel_shape: torch.Size([64, 3, 3, 3])
[ 1]        layer1.0.conv1 -- kernel_shape: torch.Size([64, 64, 1, 1])
[ 2]        layer1.0.conv2 -- kernel_shape: torch.Size([64, 64, 3, 3])
[ 3]        layer1.0.conv3 -- kernel_shape: torch.Size([256, 64, 1, 1])
[ 3] layer1.0.downsample.0 -- kernel_shape: torch.Size([256, 64, 1, 1])
[ 4]        layer1.1.conv1 -- kernel_shape: torch.Size([64, 256, 1, 1])
[ 5]        layer1.1.conv2 -- kernel_shape: torch.Size([64, 64, 3, 3])
[ 6]        layer1.1.conv3 -- kernel_shape: torch.Size([256, 64, 1, 1])
[ 7]        layer1.2.conv1 -- kernel_shape: torch.Size([64, 256, 1, 1])
[ 8]        layer1.2.conv2 -- kernel_shape: torch.Size([64, 64, 3, 3])
[ 9]        layer1.2.conv3 -- kernel_shape: torch.Size([256, 64, 1, 1])
[10]        layer2.0.conv1 -- kernel_shape: torch.Size([128, 256, 1, 1])
[11]        layer2.0.conv2 -- kernel_shape: torch.Size([128, 128, 3, 3])
[12]        

EPOCH NUMBER  0


 81%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                              | 571/704 [01:09<00:16,  8.12it/s, loss=3.92]